In [70]:
import rospy
import rospkg
import json
from baxter_commander.persistence import statetodict
from baxter_commander import ArmCommander
from scipy.io import loadmat
from os.path import join
from numpy import array

In [2]:
rospy.init_node('record_home_pose')

In [64]:
right = ArmCommander('right', fk='kdl')

Unknown tag: materialUnknown tag: materialScalar element defined multiple times: collisionScalar element defined multiple times: collision

In [4]:
rospack = rospkg.RosPack()

In [7]:
pkg_action = rospack.get_path('thr_action_server')
pkg_reba = rospack.get_path('reba_optim')

In [46]:
# open config mat
conf_file = join(pkg_reba, 'matlab_source', 'matlab_toy_insertion', 'elbowInitConfig.mat')
conf_dict = loadmat(conf_file)
joint_values = list(conf_dict['elbow']['down'][0][0]['qRest'][0][0][0])

[WARN] [WallTime: 1472639235.218869] Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.
[WARN] [WallTime: 1472639235.220132] Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.


In [79]:
output_dict = {}

In [80]:
current_state = right.get_current_state()

In [81]:
# replace state value by config ones and convert to dict
current_state.joint_state.position = joint_values
output_dict['state'] = statetodict(current_state)

In [82]:
# calculate the corresponding gripper pose
fk = right.get_fk(robot_state=current_state)
output_dict['pose'] = [fk[0].tolist(), fk[1].tolist()]

In [83]:
# write the config file
with open(join(pkg_action, 'config', 'shapeo_home_pose.json'), 'w') as outfile:
    json.dump(output_dict, outfile)